In [ ]:
!nvidia-smi

Sun Mar  2 14:25:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   66C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install imbalanced-learn


In [ ]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.9 MB/s eta 0:00:00


In [ ]:
%pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.4 MB/s eta 0:00:00


In [ ]:
! pip install nltk

In [ ]:
!pip install inflect


In [ ]:
import gc
import torch
import pandas as pd
import ast  # for list-like labels, if needed
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from transformers import (
    DebertaTokenizer,
    DebertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)
from evaluate import load
import numpy as np
import optuna  # Using optuna for hyperparameter optimization

import nltk
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize
import random
import inflect

In [ ]:
# Clean up GPU memory
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/NLP-Coursework/

/content/drive/MyDrive/NLP-Coursework


In [ ]:
# -------------------------------
# Data Preparation and Tokenization + ADDED RANDOM WORD DELETION
# -------------------------------

import random
import re
import torch
import pandas as pd
from transformers import DebertaTokenizer, DebertaForSequenceClassification

# Define column names and load datasets
columns = ['par_id', 'art_id', 'keyword', 'country_code', 'text', 'label']
df_texts = pd.read_csv("dontpatronizeme_pcl.tsv", sep="\t", header=None, names=columns)
df_labels = pd.read_csv("train_semeval_parids-labels.csv")

# Ensure the IDs are strings for merging
df_texts["par_id"] = df_texts["par_id"].astype(str)
df_labels["par_id"] = df_labels["par_id"].astype(str)

# Drop unnecessary label column from df_labels and create binary labels in df_texts
df_labels = df_labels.drop(columns=["label"])
df_texts["binary_label"] = df_texts["label"].apply(lambda x: 1 if x >= 2 else 0)
df_texts = df_texts.drop(columns=["label"])

# Merge datasets on paragraph ID and rename the binary label to "label"
df = df_labels.merge(df_texts, on="par_id", how="left")
df.rename(columns={"binary_label": "label"}, inplace=True)
df = df.dropna(subset=["text", "label"])  # Drop rows with missing data

def random_deletion(text, p=0.05):
    """
    Randomly deletes words from the input text with probability p.
    If all words are deleted, returns one random word to ensure non-empty text.
    """
    words = text.split()
    if len(words) == 0:
        return text
    # Keep words with probability (1-p)
    new_words = [word for word in words if random.random() > p]
    # Ensure at least one word remains
    if len(new_words) == 0:
        new_words = [random.choice(words)]
    return " ".join(new_words)

def clean_text(text, augment=False):
    """
    Cleans the text and applies random word deletion augmentation if specified.
    """
    text = re.sub(r"\s+", " ", text)  # Normalize whitespace
    if augment:
        text = random_deletion(text, p=0.05)  # Delete words with 20% probability
    return text.strip()

# Apply the function to augment a fraction of the dataset (50% chance for each example)
df["text"] = df["text"].astype(str).apply(lambda x: clean_text(x, augment=random.random() < 0.5))

print("Preprocessing and balancing complete!")

# Initialize the tokenizer and model
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")
model = DebertaForSequenceClassification.from_pretrained(
    "microsoft/deberta-base", num_labels=2, ignore_mismatched_sizes=True
)

# Tokenization function
def tokenize_function(text):
    return tokenizer(text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")

# Tokenize the texts and stack tensors
df["tokenized"] = df["text"].apply(lambda x: tokenize_function(x))
input_ids = torch.cat([t["input_ids"] for t in df["tokenized"]], dim=0)
attention_masks = torch.cat([t["attention_mask"] for t in df["tokenized"]], dim=0)
labels = torch.tensor(df["label"].values, dtype=torch.long)


Preprocessing and balancing complete!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
## HERE WE CAN CHECK HOW SENTENCE GOT AUGMENTED

sample_row = df.sample(1)  # Get a random row
original_text = sample_row["text"].values[0]  # Original (possibly augmented) text
unaugmented_text = clean_text(original_text, augment=False)  # Revert to unaugmented
augmented_text = clean_text(original_text, augment=True)  # Reapply augmentation

print("\nOriginal (Before Augmentation):", unaugmented_text)
print("Augmented Text:", augmented_text)



Original (Before Augmentation): The weaker sex ? Science that shows women are stronger than men
Augmented Text: The weaker sex ? Science that shows women are stronger than men


In [ ]:
# -------------------------------
# Create Dataset and Sampler
# -------------------------------

class PCLDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_masks[idx],
            "labels": self.labels[idx],
        }

# Split data into training and validation sets using indices
indices = list(range(len(df)))
train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)

train_ids, val_ids = input_ids[train_idx], input_ids[val_idx]
train_masks, val_masks = attention_masks[train_idx], attention_masks[val_idx]
train_labels, val_labels = labels[train_idx], labels[val_idx]

train_dataset = PCLDataset(train_ids, train_masks, train_labels)
val_dataset = PCLDataset(val_ids, val_masks, val_labels)


In [ ]:
# -------------------------------
# Define Metric Computation
# -------------------------------

def compute_metrics(eval_pred):
    """Computes Accuracy and F1 Score"""
    accuracy_metric = load("accuracy")
    f1_metric = load("f1")
    predictions = np.argmax(eval_pred.predictions, axis=1)
    references = eval_pred.label_ids
    accuracy_score = accuracy_metric.compute(predictions=predictions, references=references)
    f1_score = f1_metric.compute(predictions=predictions, references=references)
    return {"accuracy": accuracy_score["accuracy"], "f1": f1_score["f1"]}


In [ ]:
# -------------------------------
# Define the Hyperparameter Tuning Objective with Early Stopping
# -------------------------------

def objective(trial):
    # Suggest hyperparameters (updated to use suggest_float with log=True)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0, 0.3)

    # Update TrainingArguments with trial hyperparameters
    training_args = TrainingArguments(
       output_dir="./results",
       learning_rate=learning_rate,
       optim="adamw_torch",
       warmup_ratio=0.1,
       evaluation_strategy="epoch",
       save_strategy="no", # change to no
       per_device_train_batch_size=4,  # Reduce from 8 or 16 to 4 or even 2
       per_device_eval_batch_size=4,   # Match train batch size
       num_train_epochs=2,
       weight_decay=weight_decay,
       logging_dir="./logs",
       logging_steps=50,
       save_total_limit=2,
       lr_scheduler_type="linear",
       load_best_model_at_end=False,
       metric_for_best_model="f1",
       report_to="none",
       fp16=False,  # Enables mixed precision training (reduces memory usage)
       bf16=False,  # Keep this False unless on Ampere GPUs (A100, RTX 30xx)
   )


    # Initialize Trainer with early stopping callback
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    # Train and evaluate the model
    trainer.train()
    eval_result = trainer.evaluate()

    # Use F1 score as the objective to maximize
    f1 = eval_result["eval_f1"]
    print(f"Trial completed with F1: {f1}")
    return f1

# -------------------------------
# Run Hyperparameter Tuning with Optuna
# -------------------------------

# Create a study that maximizes the F1 score
study = optuna.create_study(direction="maximize")
# Run the optimization for a fixed number of trials (e.g., 10)
study.optimize(objective, n_trials=4)

# Print the best hyperparameters found
print("Best hyperparameters:", study.best_trial.params)

[I 2025-02-28 21:58:35,053] A new study created in memory with name: no-name-f3c6d2c4-481e-49d7-8843-12976bbcff9e
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.342300,0.298426,0.912836,0.365217
2,0.222100,0.369062,0.915224,0.470149


[I 2025-02-28 22:38:01,674] Trial 0 finished with value: 0.4701492537313433 and parameters: {'learning_rate': 1.3462263285933991e-05, 'weight_decay': 0.15493888050568264}. Best is trial 0 with value: 0.4701492537313433.


Trial completed with F1: 0.4701492537313433


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Trial completed with F1: 0.49169435215946844


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.000500,0.828011,0.914030,0.510204
2,0.156000,0.820587,0.908657,0.495050


[I 2025-03-01 00:36:24,268] Trial 3 finished with value: 0.49504950495049505 and parameters: {'learning_rate': 8.210675025111941e-06, 'weight_decay': 0.060791107835599954}. Best is trial 1 with value: 0.5068493150684932.


Trial completed with F1: 0.49504950495049505
Best hyperparameters: {'learning_rate': 7.99102729694755e-06, 'weight_decay': 0.1257050111683509}


In [ ]:


# -------------------------------
# Final Training with Best Hyperparameters and Early Stopping
# -------------------------------

final_training_args = TrainingArguments(
       output_dir="./results",
       learning_rate=7.99102729694755e-06,
       optim="adamw_torch",
       warmup_ratio=0.1,
       evaluation_strategy="epoch",
       save_strategy="no", # change to no
       per_device_train_batch_size=4,  # Reduce from 8 or 16 to 4 or even 2
       per_device_eval_batch_size=4,   # Match train batch size
       num_train_epochs=2,
       weight_decay=0.1257050111683509,
       logging_dir="./logs",
       logging_steps=50,
       save_total_limit=2,
       lr_scheduler_type="linear",
       load_best_model_at_end=False,
       metric_for_best_model="f1",
       report_to="none",
       fp16=False,  # Enables mixed precision training (reduces memory usage)
       bf16=False,  # Keep this False unless on Ampere GPUs (A100, RTX 30xx)
   )

final_trainer = Trainer(
    model=model,
    args=final_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics

)

final_trainer.train()
final_results = final_trainer.evaluate()
print("Final evaluation results:", final_results)

# Save the final model and tokenizer
model.save_pretrained("./deberta_pcl_model")
tokenizer.save_pretrained("./deberta_pcl_model")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.333100,0.285321,0.915821,0.384279
2,0.166600,0.346943,0.920000,0.503704


Final evaluation results: {'eval_loss': 0.34694308042526245, 'eval_accuracy': 0.92, 'eval_f1': 0.5037037037037037, 'eval_runtime': 90.7978, 'eval_samples_per_second': 18.448, 'eval_steps_per_second': 4.615, 'epoch': 2.0}


('./deberta_pcl_model/tokenizer_config.json',
 './deberta_pcl_model/special_tokens_map.json',
 './deberta_pcl_model/vocab.json',
 './deberta_pcl_model/merges.txt',
 './deberta_pcl_model/added_tokens.json')

In [ ]:
columns = ['par_id', 'art_id', 'keyword', 'country_code', 'text', 'label']
def tokenize_function(text):
    return tokenizer(text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")

df_dev = pd.read_csv("dev_semeval_parids-labels.csv")
df_texts = pd.read_csv("dontpatronizeme_pcl.tsv", sep="\t", header=None, names=columns)

df_dev["par_id"] = df_dev["par_id"].astype(str)
df_texts["par_id"] = df_texts["par_id"].astype(str)

df_dev = df_dev.drop(columns=["label"])
df_texts["binary_label"] = df_texts["label"].apply(lambda x: 1 if x >= 2 else 0)
df_texts = df_texts.drop(columns=["label"])

# Merge datasets on paragraph ID and rename the binary label to "label"
df_dev = df_dev.merge(df_texts, on="par_id", how="left")
df_dev.rename(columns={"binary_label": "label"}, inplace=True)

#DON'T DROP
#df_dev = df_dev.dropna(subset=["text", "label"])  # Drop rows with missing data
df_dev["text"] = df_dev["text"].fillna("")

print(len(df_dev))

def clean_text(text):
    text = re.sub(r"\s+", " ", text)  # Remove extra spaces
    #text = re.sub(r"[^a-zA-Z0-9.,!?'\"]", " ", text)  # Remove special characters
    return text.strip()

# Ensure text column has no NaN values before applying text cleaning
df_dev["text"] = df_dev["text"].astype(str).apply(clean_text)
def tokenize_function(text):
    return tokenizer(text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")

# Tokenize the texts and stack tensors
df_dev["tokenized"] = df_dev["text"].apply(lambda x: tokenize_function(x))

dev_input_ids = torch.cat([t["input_ids"] for t in df_dev["tokenized"]], dim=0)
dev_attention_masks = torch.cat([t["attention_mask"] for t in df_dev["tokenized"]], dim=0)
dev_labels = torch.tensor(df_dev["label"].values, dtype=torch.long)


# Create a Dataset instance (assuming you have a PCLDataset defined similarly)
dev_dataset = PCLDataset(dev_input_ids, dev_attention_masks, dev_labels)

2094


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.metrics import f1_score
import torch

# Load the saved model from the directory where you saved it
model = AutoModelForSequenceClassification.from_pretrained("./deberta_pcl_model")

# Set up the TrainingArguments for evaluation only
eval_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=4,
    report_to="none",  # Disable logging if not needed
)

# Create a Trainer instance for evaluation
trainer = Trainer(
    model=model,
    args=eval_args,
    compute_metrics=compute_metrics,
)

# Evaluate the model on your dev dataset
eval_results = trainer.evaluate(eval_dataset=dev_dataset)
print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 0.30260947346687317, 'eval_model_preparation_time': 0.0029, 'eval_accuracy': 0.9255014326647565, 'eval_f1': 0.5214723926380368, 'eval_runtime': 111.9905, 'eval_samples_per_second': 18.698, 'eval_steps_per_second': 4.679}
